In [ ]:
# Installation Library

!pip install transformers datasets sentencepiece rouge 

In [ ]:
import sys
import torch
import pandas as pd
from rouge import Rouge
from transformers import pipeline
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [ ]:
from datasets import load_dataset
dataset = load_dataset("billsum")

Using custom data configuration default


Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

Dataset billsum downloaded and prepared to /root/.cache/huggingface/datasets/billsum/default/3.0.0/d1e95173aed3acb71327864be74ead49b578522e4c7206048b2f2e5351b57959. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
train = dataset['train']
test_cases =  dataset['ca_test']

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "Frederick0291/t5-small-finetuned-billsum"

T5tokenizer = T5Tokenizer.from_pretrained(model_name)
T5model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.88k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("Frederick0291/t5-small-finetuned-billsum")
model = AutoModelForSeq2SeqLM.from_pretrained("Frederick0291/t5-small-finetuned-billsum").to(device)

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [ ]:
case1 = test_cases[0]['text']
summary1 =  test_cases[0]['summary']

In [ ]:
batch = tokenizer(case1, return_tensors="pt").to(device)

In [ ]:
summary = model.generate(**batch, max_length=1000, min_length=500)
systemsummary = tokenizer.batch_decode(summary, skip_special_tokens=True)

In [ ]:
print(systemsummary)

[', owned by a veterans’ organization that has been chartered by the Congress of the United States, organized and operated for charitable purposes, and exempt from federal income tax as an organization described in Section XIII of the California Constitution. (2) The use of real property by a veterans’ organization for fraternal, lodge, or social club purposes constitutes the exclusive use of that property for a charitable purpose within the meaning of Section XIII of the California Constitution. (2) The State Board of Equalization’s interpretation of the Revenue and Taxation Code. (2) The state shall not reimburse any local agency for any property tax revenues lost by it pursuant to this act. (2) The State Board of Equalization’s interpretation of the Revenue and Tax Code. (2) The statutes of limitations shall apply to any portion of a property used solely and exclusively for the purpose of the veterans’ organizations. (2) The statutes of limitations. (2) The statute of limitations of

In [ ]:
standard_summary = summary1
system_summary = systemsummary[0]

In [ ]:
rouge = Rouge()
score = rouge.get_scores(system_summary, standard_summary)
pd.DataFrame(score[0]).set_index([['recall','precision','f-measure']])

,rouge-1,rouge-2,rouge-l
recall,0.597938,0.402597,0.556701
precision,0.537037,0.324607,0.500000
f-measure,0.565854,0.359420,0.526829


### Test DataSet

In [ ]:
CasesText = test_cases['text']
GoldSummary = test_cases['summary']

In [ ]:
len(CasesText), len(GoldSummary)

(1237, 1237)

In [ ]:
SystemSummary = []

for i, case in enumerate(CasesText):

  batch = tokenizer(case, return_tensors="pt").to(device)
  summary = model.generate(**batch, max_length=3000, min_length=500)
  Summary = tokenizer.batch_decode(summary, skip_special_tokens=True)

  SystemSummary.append(Summary)
  print(i)

In [ ]:
len(SystemSummary), len(GoldSummary)

(1237, 1237)

In [ ]:
SystemSummaryFinal = []

for i in SystemSummary:
  SystemSummaryFinal.append((i[0]))

In [ ]:
Summaries = pd.DataFrame(list(zip(GoldSummary, SystemSummaryFinal)), columns =['GoldSummary', 'SystemSummary'])

In [ ]:
Summaries.sample(5)

,GoldSummary,SystemSummary
46,Existing law provides for the Medi-Cal program...,of the Federal Americans with Disabilities Act...
922,"Existing law, the Field Act, generally require...",", the district, county office of education, or..."
102,Existing law establishes the California Commun...,(Sec. 3) The State of California does enact as...
239,Existing law with respect to claims against pu...,of the Federal Supplemental Security Income (S...
455,Existing law prohibits the disclosure of the h...,"of a peace officer, a county sheriff’s office,..."


In [ ]:
path = "/content/drive/MyDrive/LegSuM/Data/Data by Models/"
Summaries.to_csv(path + "T5_Small.csv", header=True, index=False)

Score

In [ ]:
system_summary = Summaries['SystemSummary']
standard_summary = Summaries['GoldSummary']

In [ ]:
print(sys.getrecursionlimit())
sys.setrecursionlimit(5000)
print(sys.getrecursionlimit())

1000
5000


In [ ]:
rouge = Rouge()
score = rouge.get_scores(system_summary, standard_summary, avg=True)

In [ ]:
T5small = pd.DataFrame(score).set_index([['recall','precision','f-measure']])

In [ ]:
T5small

,rouge-1,rouge-2,rouge-l
recall,0.285932,0.132409,0.261283
precision,0.446782,0.226564,0.410743
f-measure,0.329990,0.155237,0.302192


In [ ]:
path = "/content/drive/MyDrive/LegSuM/scores/"

T5small.to_csv(path + "T5smallRouge.csv", header=True)

                                                                          The End